In [18]:
! pip install qeds
import pandas as pd
import numpy as np
!pip install pyarrow
import pyarrow.feather

# Storage Formats - Exercises

## Exercise 1

**In the cell below, the variable `url` contains a web address to a csv file containing the result of all NFL games from September 1920 to February 2017.**

1. **Use `pd.read_csv` to read this file into a DataFrame named `nfl`.** 

In [19]:
url = "https://raw.githubusercontent.com/fivethirtyeight/nfl-elo-game/"
url = url + "3488b7d0b46c5f6583679bc40fb3a42d729abd39/data/nfl_games.csv"

nfl = pd.read_csv(url, index_col=0)
nfl

,season,neutral,playoff,team1,team2,elo1,elo2,elo_prob1,score1,score2,result1
date,,,,,,,,,,,
1920-09-26,1920,0,0,RII,STP,1503.947000,1300.000000,0.824651,48,0,1.0
1920-10-03,1920,0,0,AKR,WHE,1503.420000,1300.000000,0.824212,43,0,1.0
1920-10-03,1920,0,0,RCH,ABU,1503.420000,1300.000000,0.824212,10,0,1.0
1920-10-03,1920,0,0,DAY,COL,1493.002000,1504.908000,0.575819,14,0,1.0
1920-10-03,1920,0,0,RII,MUN,1516.108000,1478.004000,0.644171,45,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...
2017-01-15,2016,0,1,DAL,GB,1617.794683,1635.451172,0.567714,31,34,0.0
2017-01-15,2016,0,1,KC,PIT,1681.926463,1647.734179,0.638993,16,18,0.0
2017-01-22,2016,0,1,ATL,GB,1664.127266,1651.537731,0.609840,44,21,1.0


2. **Print the shape and column names of `nfl`.**

In [20]:
print("nfl.shape=",nfl.shape)
print(nfl.columns)

nfl.shape= (15740, 11)
Index(['season', 'neutral', 'playoff', 'team1', 'team2', 'elo1', 'elo2',
       'elo_prob1', 'score1', 'score2', 'result1'],
      dtype='object')


3. **Save the DataFrame to a file named `nfl.xlsx`**  

In [21]:
nfl.to_excel("nfl.xlsx", "nfl")

4. **Open the spreadsheet using Excel on your computer.**  

## Exercise 2

1. **Compute the average total points in each game (note, you will need to sum two of the columns to get total points).**  

In [22]:
#we compute the total points as score1 + score 2, then we take the average
nfl["total_points"] = nfl["score1"]+nfl["score2"]
nfl["total_points"].mean()

39.95368487928844

2. **Repeat the above calculation, but only for playoff games.**  

In [23]:
#here we compute the average total points for the subset corresponding to playoff games
nfl["total_points"][nfl["playoff"]==1].mean()

42.70404411764706

3. **Compute the average score for your favorite team (you’ll need to consider when they were team1 vs team2).**   

In [24]:
#I don't have a smarter solution

#in order to get the average for AKR:
    #I sum the total points obtained by AKR either as team1 or team2
    #I sum the total number of games of AKR either as team1 or team2
    #then I divide the total points by the total number of games

sum_teams12   = nfl["total_points"][nfl["team1"]=="AKR"].sum()   + nfl["total_points"][nfl["team2"]=="AKR"].sum()
count_teams12 = nfl["total_points"][nfl["team1"]=="AKR"].count() + nfl["total_points"][nfl["team2"]=="AKR"].count()
average = sum_teams12/count_teams12
average

17.125

4. **Compute the ratio of “upsets” to total games played. An upset is defined as a team with a lower ELO winning the game.** 

In [25]:
#for all the matches 
nfl.loc[((nfl['elo1']>nfl['elo2']) & (nfl['result1']==0)) | ((nfl['elo2']>nfl['elo1']) & (nfl['result1']==1))  , 'Upset?' ] =1 
nfl.loc[(nfl['Upset?']!=1), 'Upset?']=0
nfl['Upset?'].mean()

0.34358322744599745

In [26]:
#only for the matches of AKR (as team1)
nfl.loc[((nfl['elo1']>nfl['elo2']) & (nfl['result1']==0)) | ((nfl['elo2']>nfl['elo1']) & (nfl['result1']==1))  , 'Upset?' ] =1 
nfl.loc[(nfl['Upset?']!=1), 'Upset?']=0
nfl['Upset?'][nfl["team1"] == "AKR"].mean()

0.21739130434782608